In [0]:
%%sh
# Download the CMS Open Payments ZIP file
curl -L https://download.cms.gov/openpayments/PGYR2023_P01302025_01212025.zip -o /tmp/openpayments_2023.zip

# Unzip the file to a directory, force overwriting existing files without prompting
unzip -o /tmp/openpayments_2023.zip -d /tmp/openpayments_2023


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  752M  100  752M    0     0  75.2M      0  0:00:10  0:00:10 --:--:-- 47.9M


Archive:  /tmp/openpayments_2023.zip
  inflating: /tmp/openpayments_2023/OP_PGYR2023_README_P01302025.txt  
  inflating: /tmp/openpayments_2023/OP_DTL_OWNRSHP_PGYR2023_P01302025_01212025.csv  
  inflating: /tmp/openpayments_2023/OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv  
  inflating: /tmp/openpayments_2023/OP_DTL_RSRCH_PGYR2023_P01302025_01212025.csv  
  inflating: /tmp/openpayments_2023/OP_REMOVED_DELETED_PGYR2023_P01302025_01212025.csv  


In [0]:
import os

# List the extracted files
os.listdir("/tmp/openpayments_2023")

Out[46]: ['OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv',
 'OP_DTL_RSRCH_PGYR2023_P01302025_01212025.csv',
 'OP_DTL_OWNRSHP_PGYR2023_P01302025_01212025.csv',
 'OP_PGYR2023_README_P01302025.txt',
 'OP_REMOVED_DELETED_PGYR2023_P01302025_01212025.csv']

In [0]:
# Reading the CSV file into a Spark DataFrame
df = spark.read.csv("file:/tmp/openpayments_2023/OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv", header=True, inferSchema=True)

# Saving it as a Delta table
df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("General_OpenPayments")

In [0]:
# Reading the CSV file into a Spark DataFrame
df = spark.read.csv("dbfs:/FileStore/shared_uploads/esha.yepuri@slu.edu/OP_CVRD_RCPNT_PRFL_SPLMTL_P01302025_01212025.csv", header=True, inferSchema=True)

# Saving it as a Delta table (use underscores instead of spaces)
df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("covered_recipients")


In [0]:
from pyspark.sql.functions import col, sum, count, desc

# 1. Nature of Payments with reimbursement amounts greater than $1,000, ordered by count
df_payments = spark.table("General_OpenPayments")

df_payments.filter(col("Total_Amount_of_Payment_USDollars") > 1000) \
    .groupBy("Nature_of_Payment_or_Transfer_of_Value") \
    .agg(count("*").alias("count")) \
    .orderBy(desc("count")) \
    .show(10, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------+------+
|Nature_of_Payment_or_Transfer_of_Value                                                                                                            |count |
+--------------------------------------------------------------------------------------------------------------------------------------------------+------+
|Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program|164092|
|Consulting Fee                                                                                                                                    |105228|
|Travel and Lodging                                                                                                                                |24738 |
|Honoraria                                                      

In [0]:
 #2. Top ten Nature of Payments by count
df_payments.groupBy("Nature_of_Payment_or_Transfer_of_Value") \
    .agg(count("*").alias("count")) \
    .orderBy(desc("count")) \
    .show(10, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|Nature_of_Payment_or_Transfer_of_Value                                                                                                            |count   |
+--------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|Food and Beverage                                                                                                                                 |13378464|
|Travel and Lodging                                                                                                                                |545086  |
|Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program|236628  |
|Consulting Fee                                     

In [0]:
3. #Top ten Nature of Payments by total amount
df_payments.groupBy("Nature_of_Payment_or_Transfer_of_Value") \
    .agg(sum("Total_Amount_of_Payment_USDollars").alias("total_amount")) \
    .orderBy(desc("total_amount")) \
    .show(10, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|Nature_of_Payment_or_Transfer_of_Value                                                                                                            |total_amount        |
+--------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|Royalty or License                                                                                                                                |1.1921745630200038E9|
|Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program|5.946326876500018E8 |
|Consulting Fee                                                                                                                                    |5.

In [0]:

%sql
SELECT cr.Covered_Recipient_Profile_Primary_Specialty, 
       SUM(gp.Total_Amount_of_Payment_USDollars) AS total_amount
FROM General_OpenPayments gp
JOIN covered_recipients cr 
ON gp.Covered_Recipient_NPI = cr.Covered_Recipient_NPI
GROUP BY cr.Covered_Recipient_Profile_Primary_Specialty
ORDER BY total_amount DESC
LIMIT 10;


# Top 10 Physician Specialties by Total Amount

Covered_Recipient_Profile_Primary_Specialty,total_amount
Allopathic & Osteopathic Physicians|Orthopaedic Surgery,3.7775549672999597E8
Allopathic & Osteopathic Physicians|Psychiatry & Neurology|Neurology,8.680715037000045E7
Allopathic & Osteopathic Physicians|Neurological Surgery,8.587046903000015E7
Allopathic & Osteopathic Physicians|Dermatology,8.07658914400014E7
Allopathic & Osteopathic Physicians|Orthopaedic Surgery|Adult Reconstructive Orthopaedic Surgery,7.277526618000011E7
Allopathic & Osteopathic Physicians|Internal Medicine|Hematology & Oncology,6.576093498000091E7
Allopathic & Osteopathic Physicians|Internal Medicine,6.569819551000157E7
Allopathic & Osteopathic Physicians|Psychiatry & Neurology|Psychiatry,6.548150061000022E7
null,6.473382509000038E7
Allopathic & Osteopathic Physicians|Internal Medicine|Cardiovascular Disease,6.30571815700004E7


In [0]:

 #5.Top 10 Physicians by Total Amount
df_payments.groupBy("Covered_Recipient_First_Name", "Covered_Recipient_Last_Name") \
    .agg(sum("Total_Amount_of_Payment_USDollars").alias("total_amount")) \
    .orderBy(desc("total_amount")) \
    .show(10, truncate=False)


+----------------------------+---------------------------+-------------------+
|Covered_Recipient_First_Name|Covered_Recipient_Last_Name|total_amount       |
+----------------------------+---------------------------+-------------------+
|null                        |null                       |7.933900857900007E8|
|STEPHEN                     |BURKHART                   |3.392202493E7      |
|WILLIAM                     |BINDER                     |2.943437497E7      |
|KEVIN                       |FOLEY                      |1.73059378E7       |
|IVAN                        |OSORIO                     |1.606551551E7      |
|GEORGE                      |MAXWELL                    |1.160032024E7      |
|ROBERT                      |BOOTH                      |8459167.19         |
|NEAL                        |ELATTRACHE                 |7810628.200000001  |
|AARON                       |ROSENBERG                  |6883627.29         |
|ROGER                       |JACKSON               